<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Transformer_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer のファインチューニング

元ネタ：https://qiita.com/tinymouse/items/2ab7c077c9ab56e8c13a

### 必要なライブラリのインストール

In [ ]:
!pip install transformers accelerate sentencepiece datasets

### 言語モデルの準備

In [ ]:
import transformers
import torch

# モデルとトークナイザの準備
model = transformers.AutoModelForCausalLM.from_pretrained(
    "cyberagent/open-calm-large",
    torch_dtype=torch.bfloat16
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-large"
)

### チューニング前のモデルによる生成

In [ ]:
prompt = "休日の朝は"

input = tokenizer(
    prompt,
    return_tensors="pt"
)

outputs = model.generate(
    **input,
    max_length=64,
    do_sample=True,
    num_return_sequences=5
)

for i in range(len(outputs)):
    print("%d:" % (i+1))
    output = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(output)

### 学習データの用意
参考：HuggingFace Datasets の使い方｜npaka

In [ ]:
import datasets

# データセットを用意
datadic = datasets.load_dataset("federerjiang/dialect.osaka")
dataset = datadic['train']

if len(dataset) > 1000: dataset = dataset.take(1000)  # データ量を減らす

### データセットの確認

In [ ]:
# データセットを確認
print(dataset)

# pandas 形式に変換
dataset.set_format(type="pandas")
print(dataset[:])

## チューニングの実行

In [ ]:
dataset['sentence'].to_list()

In [ ]:
tokenized_dataset = dataset.map(
    lambda example:
        tokenizer(example['sentence'].to_list(), truncation=True), batched=True
)

In [ ]:
import wandb
wandb.init(mode="disabled")

# トレーナーの準備
trainer = transformers.Trainer(
    model=model,
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False
    ),
    args=transformers.TrainingArguments(
        output_dir="./output",
        num_train_epochs=5,
        per_device_eval_batch_size=1,
    ),
    train_dataset=tokenized_dataset
)

# トレーニングする
trainer.train()

#　保存する
trainer.save_model("./trained_model")

### チューニングしたモデルによる生成

In [ ]:
# モデルとトークナイザの準備
model = transformers.AutoModelForCausalLM.from_pretrained(
    "./trained_model"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "cyberagent/open-calm-large"
)

prompt = "休日の朝は"

input = tokenizer(
    prompt,
    return_tensors="pt"
)

outputs = model.generate(
    **input,
    max_length=64,
    do_sample=True,
    num_return_sequences=5
)

for i in range(len(outputs)):
    print("%d:" % (i+1))
    output = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(output)